#Algorithms Tutorial
Once we have data for GST, there are several *algorithms* we can run on it to produce tomographic estimates. Depending on the amount of data you have, and time available for running Gate Set Tomography, one algorithm may be preferable over the others.

Currently, `pygsti` provides support for the following GST algorithms:

* **Linear Gate Set Tomography (LGST)**: Uses short gate sequences to quickly compute a rough (low accuracy) estimate of a gate set by linear inversion.

* **Extended Linear Gate Set Tomography (eLGST or EXLGST)**: Minimizes the sub-of-squared errors between independent LGST estimates and the estimates obtained from a single gate set to find a best-estimate gate set.  This is typically done in an interative fashion, using LGST estimates for longer and longer sequences. 

* **Minimum-$\chi^2$ Gate Set Tomography (MC2GST)**: Minimizes the $\chi^{2}$ statistic of the data frequencies and gate set probabilities to find a best-estimate gate set.  Typically done in an interative fashion, using successively larger sets of longer and longer gate sequences. 

* **Maximum-Likelihood Gate Set Tomography (MLGST)**: Maximizes the log-likelihood statistic of the data frequencies and gate set probabilities to find a best-estimate gate set.  Typically done in an interative fashion similar to MC2GST.  This maximum likelihood estimation (MLE) is very well-motivated from a statistics standpoint and should be the **most accurate among the algorithms**. 

If you're curious, the implementation of the algorithms for LGST, EXLGST, MC2GST, and MLGST may be found in the `pygsti.algorithms.core` module.  In this tutorial, we'll show how to invoke each of these algorithms.

###  Setup
The ingredients needed to as input to the GST algorithms are:
- a "target" `GateSet` which defines the desired gates.  This gate set is used by LGST to specify the various gate, state preparation, POVM effect, and SPAM labels, as well as to provide an initial guess for the *gauge* degrees of freedom.
- a `DataSet` containing the data that GST attempts to fit using the probabilities generated by a single `GateSet`.  This data set must at least contain the data for the gate sequences required by the algorithm that is chosen.
- for EXLGST, MC2GST, and MLGST, a list-of-lists of `GateString` objects, which specify which gate strings are used during each iteration of the algorithm (the length of the top-level list defines the number of interations).  Note that which gate strings are included in these lists is different for EXLGST than it is for MC2GST and MLGST.

In [1]:
import pygsti
import json

In [2]:
#Load target gateset, dataset, and list of fiducial gate strings.
#In this case we load directly from files created in past tutorials

gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
dsLowCounts = pygsti.io.load_dataset("tutorial_files/Example_Dataset_LowCnts.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

depol_gateset = gs_target.depolarize(gate_noise=0.1)

#Could also load a fiducial dictionary file like this:
#fiducialDict = GST.load_gatestring_dict("Example_FiducialList.txt")
#fiducialList = fiducialDict.values() #all we really need are the fiducial strings themselves

print "Loaded target gateset with gate labels: ",gs_target.gates.keys()
print "Loaded fiducial list of length: ",len(fiducialList)
print "Loaded dataset of length: ",len(ds)

Loading tutorial_files/Example_Dataset_LowCnts.txt: 100%


Writing cache file (to speed future loads):  tutorial_files/Example_Dataset_LowCnts.txt.cache
Loaded target gateset with gate labels: 

 ['Gi', 'Gx', 'Gy']
Loaded fiducial list of length:  6
Loaded dataset of length:  2737


### Using LGST to get an initial estimate

An important and distinguising property of the LGST algorithm is that it does *not* require an initial-guess `GateSet` as an input.  It uses linear inversion and short sequences to obtain a rough gate set estimate.  As such, it is very common to use the LGST estimate as the initial-guess starting point for more advanced forms of GST.

In [3]:
#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds

#create "spam specs" from the list of fiducial gate strings.  See the gate strings tutorial for more info.
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)

#run LGST
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP, guaranteeing that the gates are CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

In [4]:
print gs_lgst

rho0 =    0.7096  -0.0234   0.0264   0.7466


E0 =    0.6872   0.0084  -0.0009  -0.6489


Gi = 
   0.9955  -0.0094   0.0005  -0.0030
   0.0045   0.9273  -0.0507   0.0053
  -0.0160   0.0100   0.8971  -0.0237
  -0.0009  -0.0026  -0.0196   0.9062


Gx = 
   1.0021  -0.0017   0.0021  -0.0004
  -0.0097   0.9089  -0.0134   0.0030
  -0.0141  -0.0198  -0.0171  -0.9948
  -0.0563   0.0285   0.8119   0.0049


Gy = 
   0.9932  -0.0041   0.0154  -0.0084
   0.0427   0.0114  -0.0031   0.9818
  -0.0087   0.0059   0.8893   0.0201
  -0.0624  -0.8159   0.0134  -0.0042





### Extended LGST (eLGST or EXLGST)
EXLGST requires a list-of-lists of gate strings, one per iteration.  The elements of these lists are typically repetitions of short "germ" strings such that the final strings does not exceed some maximum length.  We created such lists in the gate string tutorial.  Now, we just load these lists from the text files they were saved in.

In [5]:
#Get rho and E specifiers, needed by LGST
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
elgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_eLGSTlist%d.txt" % l) for l in maxLengthList]
           
#run EXLGST.  The result, gs_exlgst, is a GateSet containing the estimated quantities
gs_exlgst = pygsti.do_iterative_exlgst(ds, gs_clgst, specs, elgstListOfLists, targetGateset=gs_target,
                                  svdTruncateTo=4, verbosity=2)


--- Iterative eLGST: Beginning iter 1 of 10 : 3 gate strings ---


--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 1.25306e-32
   frobenius distance to target =  0.0573675143573

--- Iterative eLGST: Beginning iter 2 of 10 : 4 gate strings ---


--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 1.74584e-30
   frobenius distance to target =  0.0573675143573

--- Iterative eLGST: Beginning iter 3 of 10 : 8 gate strings ---


--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 0.0146807
   frobenius distance to target =  0.0577518727387

--- Iterative eLGST: Beginning iter 4 of 10 : 18 gate strings ---


--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 0.0793128

...


   frobenius distance to target =  0.0575433474816

--- Iterative eLGST: Beginning iter 9 of 10 : 73 gate strings ---


--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 0.400736


   frobenius distance to target =  0.0575473797699

--- Iterative eLGST: Beginning iter 10 of 10 : 84 gate strings ---


--- eLGST (least squares) ---
  Sum of minimum least squares error (w/out reg terms) = 0.474292


   frobenius distance to target =  0.0575483601379


### Minimum-$\chi^2$ GST (MC2GST)
MC2GST and MLGST also require a list-of-lists of gate strings, one per iteration.  However, the elements of these lists are typically repetitions of short "germ" strings *sandwiched between fiducial strings* such that the repeated-germ part of the string does not exceed some maximum length.  We created such lists in the gate string tutorial.  Now, we just load these lists from the text files they were saved in.

In [6]:
#Get lists of gate strings for successive iterations of LSGST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
  
#run MC2GST.  The result is a GateSet containing the estimated quantities
gs_mc2 = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2)


--- Iterative MC2GST: Beginning iter 1 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)



--- Iterative MC2GST: Beginning iter 2 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)



--- Iterative MC2GST: Beginning iter 3 of 10 : 168 gate strings ---


...

--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 2041.43 (1969 data params - 40 model params = expected mean of 1929; p-value = 0.0369934)



--- Iterative MC2GST: Beginning iter 9 of 10 : 2353 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 2429.98 (2353 data params - 40 model params = expected mean of 2313; p-value = 0.0444223)



--- Iterative MC2GST: Beginning iter 10 of 10 : 2737 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 2799.7 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.0822636)

In [7]:
#Write the resulting EXLGST and MC2GST results to gate set text files for later reference.
pygsti.io.write_gateset(gs_exlgst, "tutorial_files/Example_eLGST_Gateset.txt","# Example result from running eLGST")
pygsti.io.write_gateset(gs_mc2,  "tutorial_files/Example_MC2GST_Gateset.txt","# Example result from running MC2GST")

In [8]:
#Run MC2GST again but use a DataSet with a lower number of counts 
gs_mc2_lowcnts = pygsti.do_iterative_mc2gst(dsLowCounts, gs_clgst, lsgstListOfLists, verbosity=2)


--- Iterative MC2GST: Beginning iter 1 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 71.5875 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371053)



--- Iterative MC2GST: Beginning iter 2 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 71.5875 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371053)



--- Iterative MC2GST: Beginning iter 3 of 10 : 168 gate strings ---


...

--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 1900.6 (1969 data params - 40 model params = expected mean of 1929; p-value = 0.673193)



--- Iterative MC2GST: Beginning iter 9 of 10 : 2353 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 2295.93 (2353 data params - 40 model params = expected mean of 2313; p-value = 0.595549)



--- Iterative MC2GST: Beginning iter 10 of 10 : 2737 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 2670.4 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.63844)

### Maximum Likelihood GST (MLGST)
Executing MLGST is very similar to MC2GST: the same gate string lists can be used and calling syntax is nearly identitcal.

In [9]:
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList] 

#run MLGST.  The result is a GateSet containing the estimated quantities
gs_mle = pygsti.do_iterative_mlgst(ds, gs_clgst, lsgstListOfLists, verbosity=2)


--- Iterative MLGST: Beginning iter 1 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)


    2*Delta(log(L)) = 48.4604

--- Iterative MLGST: Beginning iter 2 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)


    2*Delta(log(L)) = 48.4603

--- Iterative MLGST: Beginning iter 3 of 10 : 168 gate strings ---


...



--- Iterative MLGST: Beginning iter 10 of 10 : 2737 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 2799.7 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.0822636)


    2*Delta(log(L)) = 2803.55


--- Last Iteration: switching to ML objective ---
--- MLGST ---
  Maximum log(L) = 1401.41 below upper bound of -4.60013e+06


    2*Delta(log(L)) = 2802.82 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.0761427)
    2*Delta(log(L)) = 2802.82


In [10]:
#Run MLGST again but use a DataSet with a lower number of counts 
gs_mle_lowcnts = pygsti.do_iterative_mlgst(dsLowCounts, gs_clgst, lsgstListOfLists, verbosity=2)


--- Iterative MLGST: Beginning iter 1 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 71.5875 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371053)


    2*Delta(log(L)) = 73.4375

--- Iterative MLGST: Beginning iter 2 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 71.5875 (92 data params - 40 model params = expected mean of 52; p-value = 0.0371053)


    2*Delta(log(L)) = 73.4369

--- Iterative MLGST: Beginning iter 3 of 10 : 168 gate strings ---


...



--- Iterative MLGST: Beginning iter 10 of 10 : 2737 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 2670.4 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.63844)


    2*Delta(log(L)) = 2727.56


--- Last Iteration: switching to ML objective ---
--- MLGST ---
  Maximum log(L) = 1356.67 below upper bound of -228670


    2*Delta(log(L)) = 2713.34 (2737 data params - 40 model params = expected mean of 2697; p-value = 0.408603)
    2*Delta(log(L)) = 2713.34


##Compare MLGST with MC2GST

Both MLGST and MC2GST use a $\chi^{2}$ optimization procedure for all but the final iteration. For the last set of gatestrings (the last iteration), MLGST uses a maximum likelihood estimation. Below, we show how close the two estimates are to one another.  First, we optimize the gauge so the estimated gates are as close to the target gates as the gauge degrees of freedom allow.

In [11]:
# We optimize over the gate set gauge
gs_mle = pygsti.optimize_gauge(gs_mle,"target",targetGateset=depol_gateset)
gs_mle_lowcnts = pygsti.optimize_gauge(gs_mle_lowcnts,"target",targetGateset=depol_gateset)
gs_mc2 = pygsti.optimize_gauge(gs_mc2,"target",targetGateset=depol_gateset)
gs_mc2_lowcnts = pygsti.optimize_gauge(gs_mc2_lowcnts,"target",targetGateset=depol_gateset)

In [12]:
print "Frobenius diff btwn MLGST and datagen = {0}".format(round(gs_mle.frobeniusdist(depol_gateset), 6))
print "Frobenius diff btwn MC2GST and datagen = {0}".format(round(gs_mc2.frobeniusdist(depol_gateset), 6))
print "Frobenius diff btwn MLGST and LGST = {0}".format(round(gs_mle.frobeniusdist(gs_clgst), 6))
print "Frobenius diff btwn MLGST and MC2GST = {0}".format(round(gs_mle.frobeniusdist(gs_mc2), 6))
print "Chi^2( MC2GST ) = {0}".format(round(pygsti.chi2(ds, gs_mc2, lsgstListOfLists[-1]), 4))
print "Chi^2( MLGST ) = {0}".format(round(pygsti.chi2(ds, gs_mle, lsgstListOfLists[-1] ), 4))
print "LogL( MC2GST ) = {0}".format(round(pygsti.logl(gs_mc2, ds, lsgstListOfLists[-1]), 4))
print "LogL( MLGST ) = {0}".format(round(pygsti.logl(gs_mle, ds, lsgstListOfLists[-1]), 4))

Frobenius diff btwn MLGST and datagen = 0.036058
Frobenius diff btwn MC2GST and datagen = 0.036375
Frobenius diff btwn MLGST and LGST = 0.012266
Frobenius diff btwn MLGST and MC2GST = 0.000895
Chi^2( MC2GST ) = 2799.6981


Chi^2( MLGST ) = 2800.569


LogL( MC2GST ) = -4601531.806


LogL( MLGST ) = -4601531.4427

Notice that, as expected, the MC2GST estimate has a *slightly* lower $\chi^{2}$ score than the MLGST estimate, and the MLGST estimate has a *slightly* higher loglikelihood than the MC2GST estimate. In addition, _both_ are close (in terms of the Frobenius difference) to the depolarized gateset. Which is good - it means GST is giving us estimates which are close to the _true_ gateset used to generate the data.  Performing the same analysis with the low-count data shows larger differences between the two, which is expected since the $\chi^2$ and loglikelihood statistics are more similar at large $N$, that is, for large numbers of samples.

In [13]:
print "LOW COUNT DATA:"
print "Frobenius diff btwn MLGST and datagen = {0}".format(round(gs_mle_lowcnts.frobeniusdist(depol_gateset), 6))
print "Frobenius diff btwn MC2GST and datagen = {0}".format(round(gs_mc2_lowcnts.frobeniusdist(depol_gateset), 6))
print "Frobenius diff btwn MLGST and LGST = {0}".format(round(gs_mle_lowcnts.frobeniusdist(gs_clgst), 6))
print "Frobenius diff btwn MLGST and MC2GST = {0}".format(round(gs_mle_lowcnts.frobeniusdist(gs_mc2), 6))
print "Chi^2( MC2GST ) = {0}".format(round(pygsti.chi2(dsLowCounts, gs_mc2_lowcnts, lsgstListOfLists[-1]), 4))
print "Chi^2( MLGST ) = {0}".format(round(pygsti.chi2(dsLowCounts, gs_mle_lowcnts, lsgstListOfLists[-1] ), 4))
print "LogL( MC2GST ) = {0}".format(round(pygsti.logl(gs_mc2_lowcnts, dsLowCounts, lsgstListOfLists[-1]), 4))
print "LogL( MLGST ) = {0}".format(round(pygsti.logl(gs_mle_lowcnts, dsLowCounts, lsgstListOfLists[-1]), 4))

LOW COUNT DATA:
Frobenius diff btwn MLGST and datagen = 0.035885
Frobenius diff btwn MC2GST and datagen = 0.036599
Frobenius diff btwn MLGST and LGST = 0.017478
Frobenius diff btwn MLGST and MC2GST = 0.011435
Chi^2( MC2GST ) = 2670.4002


Chi^2( MLGST ) = 2684.7563


LogL( MC2GST ) = -230034.0117


LogL( MLGST ) = -230026.9026